In [16]:
import pandas as pd

#handle weird kaleido error
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.kaleido.scope.chromium_args = tuple([arg for arg in pio.kaleido.scope.chromium_args if arg != "--disable-dev-shm-usage"])

from datetime import datetime, timedelta, date
import numpy as np
import time
import os
import asyncio, aiohttp, nest_asyncio
from aiohttp_retry import RetryClient, ExponentialRetry
nest_asyncio.apply()
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:71.0) Gecko/20100101 Firefox/71.0'}

In [17]:
#https://stackoverflow.com/questions/23267409/how-to-implement-retry-mechanism-into-python-requests-library

import logging
import requests

from requests.adapters import HTTPAdapter, Retry

s = requests.Session()
retries = Retry(total=10, backoff_factor=1, status_forcelist=[ 404, 502, 503, 504 ])
s.mount('http://', HTTPAdapter(max_retries=retries))

pwd = os.getcwd()
if 'L2_Fees' in pwd:
    prepend = ''
else:
    prepend = 'L2_Fees/'

In [18]:
statuses = {x for x in range(100, 600)}
statuses.remove(200)
statuses.remove(429)
statuses.remove(500)

trailing_num_days = 180#90

start_date = date.today()-timedelta(days=trailing_num_days +1)

start_date = max(start_date, date(2021,11,12))
print(start_date)
# start_date = datetime.strptime('2022-07-13', '%Y-%m-%d').date()

2022-05-24


In [19]:
async def get_cryptostats_api(api_core_string,dt_string ):
        dt_string = dt_string.strftime("%Y-%m-%d")
        i = 0
        res = pd.DataFrame()
        retry_client = RetryClient()
        # print(api_core_string + dt_string)
        async with retry_client.get(api_core_string + dt_string, retry_options=ExponentialRetry(attempts=10), raise_for_status=statuses) as response:
                try:
                        prot_req = await response.json()
                        res = pd.json_normalize(prot_req['data']).reset_index()
                        res['date'] = dt_string
                except Exception as e:
                        raise Exception("Could not convert json")
        await retry_client.close()
        return res
        

In [20]:
# https://api.cryptostats.community/api/v1/<collection-id>/<query>/<arg>

api_core = 'https://api.cryptostats.community/api/v1/fees/oneDayFeeBreakdown/' #Can only pull everything all at once. Odd.
date_rng = pd.date_range(start=start_date, end=date.today()-timedelta(days=1))
date_rng_str = ','.join(x.strftime("%Y-%m-%d") for x in date_rng)

In [25]:
# resp = s.get ('https://api.cryptostats.community/api/v1/fees/oneDayFeeBreakdown/2022-11-18')
# print(resp.json())

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [21]:
def get_range(date_range):
        data_dfs = []
        fee_df = []
 
        loop = asyncio.get_event_loop()
        tasks = [get_cryptostats_api(api_core,dt) for dt in date_range]

        data_dfs = loop.run_until_complete(asyncio.gather(*tasks, return_exceptions=True))

        data_dfs = [value for value in data_dfs if type(value) == pd.DataFrame]
        
        # print(data_dfs)

        fee_df = pd.concat(data_dfs)
        return fee_df

In [22]:
fdf_raw = get_range(date_rng)

In [23]:
fdf = fdf_raw.copy()
fdf = fdf[(fdf['metadata.blockchain'] == 'Optimism') & (fdf['metadata.name'] == 'Aave')]
display(fdf.columns)
fdf = fdf[['date','metadata.name'
                ,'results.oneDayFeeBreakdown.treasuryIncomeUSD'
                ,'results.oneDayFeeBreakdown.outstandingTreasuryIncomeUSD'
                # ,'results.oneDayFeeBreakdown.lifetimeReserveFactorAccrued'
        ]]

display(fdf)
# fdf['results.oneDayFeeBreakdown.lifetimeReserveFactorAccrued'] = fdf['results.oneDayFeeBreakdown.lifetimeReserveFactorAccrued'].fillna(0)

# display(fdf)

Index(['index', 'id', 'bundle', 'results.oneDayFeeBreakdown',
       'metadata.source', 'metadata.icon', 'metadata.name',
       'metadata.category', 'metadata.description', 'metadata.feeDescription',
       'metadata.blockchain', 'metadata.website', 'metadata.tokenTicker',
       'metadata.tokenCoingecko', 'metadata.protocolLaunch',
       'metadata.tokenLaunch', 'metadata.bundle', 'metadata.adapter',
       'metadata.subtitle', 'metadata.shortName', 'metadata.events',
       'metadata.protocolShutdown', 'errors.oneDayFeeBreakdown',
       'metadata.tokenSymbol', 'metadata.subcategory', 'metadata.tokenticker',
       'metadata.tokencoingecko', 'date',
       'results.oneDayFeeBreakdown.depositorInterestUSD',
       'results.oneDayFeeBreakdown.flashloanLPPremiumUSD',
       'results.oneDayFeeBreakdown.flashloanProtocolPremiumUSD',
       'results.oneDayFeeBreakdown.portalLPFeeUSD',
       'results.oneDayFeeBreakdown.portalProtocolFeeUSD',
       'results.oneDayFeeBreakdown.treasuryInco

,date,metadata.name,results.oneDayFeeBreakdown.treasuryIncomeUSD,results.oneDayFeeBreakdown.outstandingTreasuryIncomeUSD
45,2022-05-24,Aave,NaN,NaN
45,2022-05-25,Aave,0.000000,27.278345
45,2022-05-26,Aave,34.757668,19.858337
45,2022-05-27,Aave,NaN,NaN
45,2022-05-28,Aave,NaN,NaN
...,...,...,...,...
45,2022-11-16,Aave,NaN,NaN
45,2022-11-17,Aave,NaN,NaN
45,2022-11-18,Aave,NaN,NaN
45,2022-11-19,Aave,NaN,NaN
